In [ ]:
import pandas as pd

import io
import re
import os

pd.options.display.max_columns = None
pd.options.display.max_rows = None

triples = pd.DataFrame(columns=['e1', 'rel', 'e2'])


In [ ]:
# grab only the lines that include data
def convert(path):
    path = 'callum_test/' + path
    all_lns = ''
    nl = ''
    with open(path, 'r') as f:
        for line in f:
            if '\t' in line:
                nl = line.rstrip()
                all_lns += nl + '\n'
                
    return(io.StringIO(all_lns))

for file in os.listdir('callum_test'):
    print(file)

    if not file.startswith('.'):
        data = pd.read_csv(convert(file), sep='\t', names=['token_id', 'offset_pos', 'text', 'entity_tag', 'relation_tag', 'related_token'])

        only_entities = data[data.entity_tag != '_']

        def full_entity(ent):
            ent_tag = only_entities.loc[only_entities.token_id == ent,'entity_tag']    
            et = ent_tag.to_string(index=False)
            
            full_ent = ''
            
            if '[' in et:
                for tag in only_entities.index:
                    if only_entities['entity_tag'][tag] == et:
                        full_ent += only_entities['text'][tag] + ' '
                return(full_ent)
            else:
                ent_txt = only_entities.loc[only_entities.token_id == ent,'text']
                ent_txt = ent_txt.to_string(index=False)
                return(ent_txt)     

        for i in only_entities.index:
            if only_entities['relation_tag'][i] != '_':
                e1 = only_entities['token_id'][i]
                e1t = full_entity(e1)
                relation = only_entities['relation_tag'][i]
                e2 = only_entities['related_token'][i]
                        
                if '|' in relation:
                    ls_rel = relation.split('|')
                    ls_e2 = e2.split('|')
                    for j in range(len(ls_rel)):
                        tid = ls_e2[j].split('[')[0]
                        e2t = full_entity(tid)
                        
                        ls_rel[j] = ls_rel[j].replace('\\', '')
                                                
                        triple = e2t.strip() + '|' + ls_rel[j].strip() + '|' + e1t.strip()
                        print(triple)


                        mk_ls = triple.split('|')
                        print(mk_ls)
                        triples.loc[len(triples)] = mk_ls    
                else:       
                    tid = e2.split('[')[0]
                    e2t = full_entity(tid)
                    e2t = re.sub(r'[^\w\s]', '', e2t).strip()
            
                    relation = relation.replace('\\', '')
                    
                    triple = e2t.strip() + '|' + relation.strip() + '|' + e1t.strip()
                    print(triple)

                    mk_ls = triple.split('|')
                    print(mk_ls)
                    triples.loc[len(triples)] = mk_ls

triples.to_csv('amp-format-test.csv', index=False, header=False)
triples